In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
df = pd.read_csv('dataset.csv')
# 划分特征和目标变量
X = df.drop(['CO2 Absorptivity'], axis=1)
y = df['CO2 Absorptivity']
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)
df.head()

,Temperature,CO2 Partial Pressure,Carbonization Time,CO2 Concentration,Particle Diameter,CaO,MgO,SiO2,Al2O3,Fe2O3,MnO,L/S,CO2 Absorptivity
0,20,2.5,0.5,99.9,199.0,39.53,7.81,17.62,8.62,16.75,2.85,0.08,6.14
1,20,1.0,96.0,20.0,6.0,42.10,6.70,16.50,3.50,21.70,4.80,0.16,10.03
2,20,1.0,96.0,20.0,6.0,42.10,6.70,16.50,3.50,21.70,4.80,0.32,15.70
3,20,30.0,48.0,99.9,10.0,42.50,4.81,31.90,13.00,0.34,0.50,1.00,1.06
4,50,30.0,48.0,99.9,10.0,42.50,4.81,31.90,13.00,0.34,0.50,1.00,1.43


In [2]:
X.columns

Index(['Temperature', 'CO2 Partial Pressure', 'Carbonization Time',
       'CO2 Concentration', 'Particle Diameter', 'CaO', 'MgO', 'SiO2', 'Al2O3',
       'Fe2O3', 'MnO', 'L/S'],
      dtype='object')

In [3]:
X.describe()

,Temperature,CO2 Partial Pressure,Carbonization Time,CO2 Concentration,Particle Diameter,CaO,MgO,SiO2,Al2O3,Fe2O3,MnO,L/S
count,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000
mean,56.493711,5.369663,11.156406,88.337673,51.922610,43.173868,6.061604,16.719434,2.939969,23.397767,3.481258,9.535094
std,30.155511,8.018941,39.758124,28.480965,46.532243,8.738549,2.180101,8.124664,3.003801,14.441538,2.302029,6.855617
min,10.000000,0.000400,0.083000,0.040000,6.000000,20.910000,1.650000,6.390000,0.380000,0.250000,0.000000,0.000000
25%,30.000000,1.000000,0.500000,99.500000,31.725000,36.500000,4.170000,11.150000,1.550000,20.700000,0.670000,5.000000
50%,50.000000,1.000000,1.000000,99.900000,53.000000,42.430000,6.080000,13.600000,1.600000,24.030000,3.400000,10.000000
75%,77.500000,6.000000,4.000000,99.900000,62.000000,51.110000,6.700000,20.900000,3.230000,35.500000,5.500000,16.000000
max,180.000000,39.000000,240.000000,100.000000,532.100000,58.100000,10.900000,32.500000,13.000000,46.930000,10.620000,30.000000


In [4]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
n_estimators = [int(x) for x in np.linspace(start = 100, stop =
2000, num = 3)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num =3)]
max_depth.append(None)
min_samples_split = [2,5]
min_samples_leaf = [1,2]
bootstrap = [True]
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



#model = xgb.sklearn.XGBRegressor()
RF_model = RandomForestRegressor()
gsearch=GridSearchCV(RF_model,param_grid=param_grid, scoring='neg_mean_squared_error',cv=5,n_jobs=-1,verbose=4)
gsearch.fit(X_train, y_train)
print(gsearch.best_score_)
print(gsearch.best_params_)
best_model= gsearch.best_estimator_

predi_new=best_model.predict(X_test)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predi_new)))
print ('r:', stats.pearsonr(y_test, predi_new))


y_xgb_pre_train = best_model.predict(X_train)
y_xgb_pred = best_model.predict(X_test)

mae_train = mean_absolute_error(y_xgb_pre_train,y_train)
mse_train = mean_squared_error(y_xgb_pre_train,y_train)
R2_train = r2_score(y_train,y_xgb_pre_train)
print('training R2 = {:.3f} '.format(R2_train))
print('training mae = {:.3f} '.format(mae_train))
print('training mse = {:.3f} '.format(mse_train))
print('training RMSE = {:.3f} '.format(np.sqrt(mse_train)))

print()

mae_test = mean_absolute_error(y_xgb_pred,y_test)
mse_test = mean_squared_error(y_xgb_pred,y_test)
R2_test = r2_score(y_test,y_xgb_pred)
print('test R2 = {:.3f} '.format(R2_test))
print('test mae = {:.3f} '.format(mae_test))
print('test mse = {:.3f} '.format(mse_test))
print('test RMSE = {:.3f} '.format(np.sqrt(mse_test)))

Fitting 5 folds for each of 96 candidates, totalling 480 fits
-18.072314745767944
{'bootstrap': True, 'max_depth': 60, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Root Mean Squared Error: 4.95864733728734
r: PearsonRResult(statistic=0.9214564797550885, pvalue=3.79567007134752e-27)
training R2 = 0.984 
training mae = 0.999 
training mse = 2.209 
training RMSE = 1.486 

test R2 = 0.846 
test mae = 2.957 
test mse = 24.588 
test RMSE = 4.959 


In [5]:
import joblib
# 保存模型
joblib.dump(best_model , 'RF.pkl')

['RF.pkl']

In [9]:
RF_model = RandomForestRegressor()
trained_model = RF_model.fit(X_train, y_train)
cross_val_RMSE = -1 * cross_val_score(RF_model, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
Average_RMSE_score = cross_val_RMSE.mean()
print("Cross validation average RMSE scores: " + str(Average_RMSE_score))
cross_val_MAE = -1 * cross_val_score(RF_model, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
Average_MAE_score = cross_val_MAE.mean()
print("Cross validation average MAE scores: " + str(Average_MAE_score))
y_RF_pre_train = RF_model.predict(X_train)
R2_train = r2_score(y_train,y_RF_pre_train)
print('training R2 = {:.3f} '.format(R2_train))

train_predict = RF_model.predict(X_train)
test_predict = RF_model.predict(X_test)
test_MAE = mean_absolute_error(y_test, test_predict)
print("Test data MAE:" + str(test_MAE))
test_RMSE = mean_squared_error(y_test, test_predict) ** 0.5
print("Test data RMSE:" + str(test_RMSE))
r2_test = r2_score(y_test, test_predict)
print(f'R2 score on the test set: {r2_test:.2f}')

Cross validation average RMSE scores: 4.590385548535109
Cross validation average MAE scores: 3.1414480678804844
training R2 = 0.981 
Test data MAE:2.9215953906250007
Test data RMSE:5.4584095390605
R2 score on the test set: 0.81


In [10]:
joblib.dump(RF_model , 'RF2.pkl')

['RF2.pkl']